<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/data_prep_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sandbox notebook mostly for data processing & exploration**

In [12]:
import pandas as pd
import numpy as np
import os

In [7]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train_data.dat to train_data.dat


In [ ]:
!unzip raw_data.zip

In [5]:
# Load Dataframe Test Set: assume nomad.csv as input file name
df = pd.read_csv('raw-data.csv',  encoding="latin_1")
df.head(3)

,doc.id,title,citeulike.id,raw.title,raw.abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."


In [60]:
# raw text is the only feature being used with doc.id / index used for connecting to other data sources
df_final=df.drop(columns=['raw.title', 'citeulike.id'])
df_final['combined_text'] = df_final['title'].str.cat(df_final['raw.abstract'],sep=" ")

In [64]:
df_final.iloc[0]['combined_text']

"the metabolic world of escherichia coli is not small To elucidate the organizational and evolutionary principles of the metabolism of living organisms, recent studies have addressed the graph-theoretic analysis of large biochemical networks responsible for the synthesis and degradation of cellular building blocks [Jeong, H., Tombor, B., Albert, R., Oltvai, Z. N. \\& Barab\\{\\'a\\}si, A. L. (2000) Nature 407, 651-654; Wagner, A. \\& Fell, D. A. (2001) Proc. R. Soc. London Ser. B 268, 1803-1810; and Ma, H.-W. \\& Zeng, A.-P. (2003) Bioinformatics 19, 270-277]. In such studies, the global properties of the network are computed by considering enzymatic reactions as links between metabolites. However, the pathways computed in this manner do not conserve their structural moieties and therefore do not correspond to biochemical pathways on the traditional metabolic map. In this work, we reassessed earlier results by digitizing carbon atomic traces in metabolic reactions annotated for Escheri

**getting user embeddings**<br> The idea is to represent user via papers that s(h)e likes

In [13]:
#Helper
def load_user_article_likes(
    data_folder: str, num_users: int = 5551, num_articles: int = 13584
) -> np.ndarray:
    """
    Load the article likes for each user
    Keyword arguments:
        data_folder -- the path to the folder that contains the data
        num_users -- the number of users in the dataset - default is 5551
        num_articles -- the number of articles in the dataset - default is 13584
    Returns:
        2d numpy matrix with describing user like information.
        This matrix is num_users X num_articles.
        There is a 1 in locations in which a user likes an article u -> a
        citations[u][a] and 0 otherwise.
    """
    user_items = np.zeros([num_users, num_articles])
    with open(os.path.join(data_folder, "train_data.dat")) as users_file:
        for i, line in enumerate(users_file.readlines()):
            for article in line.strip().split()[1:]:
                user_items[i][int(article)] = 1
    return user_items

In [16]:
res = load_user_article_likes('')

In [21]:
res.shape

(5551, 13584)

In [107]:
# find the indices of the papers users like (sort of cheating... to get to lower dimensionality without
# doing anything)
user_likes = []
max_len = 0
for i in range(res.shape[0]):
  user_likes.append(np.where(res[i] == 1)[0].tolist())
  if len(user_likes[i])>max_len:
    max_len = len(user_likes[i])

In [176]:
# creating a normalized array (normalizing on the largest index) and padding to the maximal number of papers liked 
# by any user (happens to be 346)
user_likes_padded = []
for item in user_likes:
  if len(item)>0:
    max_v = max(item)
    item = [el/max_v for el in item]
  if len(item)<max_len:
    tmp = max_len-len(item)
    user_likes_padded.append(item+[0]*tmp)
  else:
    user_likes_padded.append(item)

user_likes_padded = np.array(user_likes_padded)

In [178]:
user_likes_padded.shape

(5551, 346)

In [ ]:
user_likes_padded[10]

**creating input data set** <br> The idea here is:
1. Get user profile (bunch of indices of the papers tha user likes) normilized
2. Pair it up with embeddings of those papers (title+text using Google universal sentence encoder)
3. (1)+(2) = positive examples
4. Sample raw data for the ~ same number of negative examples(a hack that uses the random number, not the best but should work relatively well giving the count)

In [125]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
 training_set = []
 for ind, item in enumerate(user_likes):
   for el in item:
    text = df_final.iloc[item]['combined_text'] #picking the text of the liked review
    embeddings = embed([text]).numpy()
    training_set.append(user_likes_padded[ind],embeddings,1)
  for i in range(len(item)): # negative samples
    random_number = np.random.randint(13584)
    if random_number not in item:
      embeddings_negative = embed([text]).numpy()
      training_set.append((user_likes_padded[ind],embeddings,0))

In [ ]:
inputs = [np.concatenate(item[0], item[1]), axis=None) for item in training_set]
targets = [item[2] for item in training_set]

**Dataset & Dataloade**<br>
*work in progress*

In [ ]:
class DATA(tr_data.Dataset):
  def __init__(inputs, targets):
    self.inputs = inputs
    self.targets = targets

  def __len__(self):
    return len(self.user_emb)

  def __getitem__(self, i):
    return {
      'input': torch.tensor(self.inputs[i], dtype = torch.float),
      'targets': torch.tensor(self.targets[i], dtype=torch.long)
    }

In [ ]:
def create_data_loader(X, y, batch_size):
  ds = DATA(
    inputs=X,
    targets=y,
  )
  return tr_data.DataLoader(
    ds,
    batch_size=batch_size,
    shuffle=True
  )
BATCH_SIZE = 128
MAX_LEN = 32
train_data_loader = create_data_loader(X_train_txt,y_train_txt, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(X_test_txt,y_test_txt, tokenizer, MAX_LEN, 189)

**Pytorch MLP**

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#input dimensons:
input_dim = 858

In [52]:
class MLP(nn.Module):

  def __init__(self):
    super(MLP, self).__init__()
    self.layer1 = nn.Linear(858, 256)
    self.layer2 = nn.Linear(256, 16)
    self.layer3 = nn.Linear(16,2)

  def forward(self, x):
    x = F.relu(self.layer1(x))
    x = F.relu(self.layer2(x))
    x = self.layer3(x)
    x = nn.Sigmoid()
    return x

net = MLP()
print(net)

MLP(
  (layer1): Linear(in_features=858, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=2, bias=True)
)


In [ ]:
criterion = nn.BCELoss()

In [ ]:
#Training Loop
loss_list = []
epoch_number = 10
for epoch in range(epoc_number):
    for batch in train_data_loader:
        optim.zero_grad()
        inputs = batch['inputs'].to(device)
        labels = batch['targets'].to(device)
        outputs = model(inputs, labels)
        loss = criterion(outputs, labels)
        acc = torch.sum((outputs[1].argmax(-1) == labels).float())/128
        loss.backward()
        optim.step()